_____
***

<img width='700' src="https://user-images.githubusercontent.com/8030363/108961534-b9a66980-7634-11eb-96e2-cc46589dcb8c.png" style="vertical-align:middle">

***
***

**Author:** [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Current Release:** **[`v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Objective:** Knowledge graphs provide meaningful ways to integrate heterogeneous biological data and represent complex biological mechanisms. This work seeks to explore the utility of incorporating existing knowledge of molecular mechanism from ontologies, publicly available data, and the literature to construct a biomedical knowledge graph that models unbiased molecular mechanisms of human disease.

<a target="_blank" href="https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png"> <img src="https://user-images.githubusercontent.com/8030363/103158881-11813b00-4780-11eb-8b45-5063765e7645.png"></a> 

(*Click Figure to Enlarge Image in Current Browser Tab*)

<br>

***
***

## Notebook Purpose
**Wiki Page:** **[`Release v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Purpose:** This notebook serves as a `main` file for the PheKnowLator project. This scripts walks through this program step-by-step and generates the knowledge graph shown above. There is also a command line version of this file ([`main.py`](https://github.com/callahantiff/PheKnowLator/blob/master/main.py)). Please see the [README](https://github.com/callahantiff/PheKnowLator/blob/master/README.md) for more information.

<br>

**Assumptions:**     
1. Make sure that the following input documents have been constructed (see the [Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) for more information):  
  - [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/edge_source_list.txt)   

2. Prepare [relations](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) and [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) files prior to running the scripts.  

3. Select a knowledge graph build type (i.e. `full`, `partial`, or `post-closure`) and construction method (i.e. `instance-based` or `subclass-based`).  

<br>

***
### Table of Contents
***
The three primary steps involved in building a knowledge graph are `Downloading Data Sources`, `Creating Edge Lists`, and `Building the knowledge graphs`.

* [Data Sources](#data-source)  
* [Create Edge Lists](#create-edges)  
* [Build Knowledge Graph](#build-kg)  

***

***

_____
### Set-Up Environment

In [31]:
# import needed libraries
import glob
import json
import pandas
import psutil
import ray
import time

# import module
from pkt_kg.downloads import OntData, LinkedData
from pkt_kg.edge_list import CreatesEdgeList
from pkt_kg.knowledge_graph import FullBuild, PartialBuild, PostClosureBuild

***
## Download Data Sources <a class="anchor" id="data-source"></a>

**Wiki Page:** **[`Dependencies`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies)**  

**Purpose:**
This portion of this portion of the algorithm is to download:
1. [Download Ontology Data](#download-ontology-data)  
2. [Download Edge Data](#download-edge-data)   

<br>

**Input Files:**
  - [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/edge_source_list.txt)

<br>

**Assumptions:**  
- All sources used to construct our knowledge graph need to be preprocessed and ready to download prior to running this code. All mapping, filtering, and label data have been generated prior to this step. For assistance with creating these datasets, see the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.  
- All downloaded and generated data sources for all PheKnowLator builds can be accessed through [Zenodo](https://zenodo.org/doi/10.5281/zenodo.7030039).


***
***
### Ontology Data  <a class="anchor" id="download-ontology-data"></a>
Ontologies are the core data structure used when building PheKnowLator. In the figure above, ontology data are shown in yellow boxes.

In [32]:
ont = OntData('resources/ontology_source_list.txt', 'resources/resource_info.txt')
#ont.downloads_data_from_url()

In [33]:
ont.parses_resource_file()

In [34]:
ont.source_list

{'disease': 'resources/ontologies/mondo_with_imports.owl',
 'napdichem': 'resources/ontologies/chebi_lite_merged_with_imports.owl',
 'protein': 'resources/ontologies/pr_with_imports.owl',
 'pathway': 'resources/ontologies/pw_with_imports.owl',
 'relation': 'resources/ontologies/ro_with_imports.owl',
 'go': 'resources/ontologies/go_with_imports.owl',
 'chemical': 'resources/ontologies/chebi_with_imports.owl',
 'phenotype': 'resources/ontologies/hp_with_imports.owl',
 'anatomy': 'resources/ontologies/uberon_with_imports.owl',
 'cell': 'resources/ontologies/clo_with_imports.owl',
 'genomic': 'resources/ontologies/so_with_imports.owl',
 'oae': 'resources/ontologies/oae_merged_with_imports.owl',
 'plant': 'resources/ontologies/po_with_imports.owl',
 'dideo': 'resources/ontologies/dideo_with_imports.owl'}

In [35]:
ont.data_files = ont.source_list
ont.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 14/14 [00:00<00:00, 56461.78it/s]


In [36]:
ont._writes_source_metadata_locally()

100%|██████████| 14/14 [00:00<00:00, 37957.50it/s]


In [37]:
ont.resource_info

['chemical-disease|:;MESH_;|class-class|RO_0002610|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;4|0:./resources/processed_data/MESH_CHEBI_MAP.txt;1:./resources/processed_data/DISEASE_MONDO_MAP.txt|5;==;marker/mechanism|None',
 "chemical-gene|;MESH_;|class-entity|RO_0002434|http://purl.obolibrary.org/obo/|http://www.ncbi.nlm.nih.gov/gene/|t|1;4|0:./resources/processed_data/MESH_CHEBI_MAP.txt|9;affects;not in x|6;==;Homo sapiens::5;.startswith('gene');",
 'chemical-gobp|:;MESH_;GO_|class-class|RO_0002436|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;5|0:./resources/processed_data/MESH_CHEBI_MAP.txt|8;<=;1.04e-47|3;==;Biological Process',
 'chemical-gocc|:;MESH_;GO_|class-class|RO_0002436|http://purl.obolibrary.org/obo/|http://purl.obolibrary.org/obo/|t|1;5|0:./resources/processed_data/MESH_CHEBI_MAP.txt|8;<=;1.04e-47|3;==;Cellular Component',
 'chemical-gomf|:;MESH_;GO_|class-class|RO_0002436|http://purl.obolibrary.org/obo/|http://purl.obolibr

<br>

### Edge Data   <a class="anchor" id="download-edge-data"></a>
In PheKnowLator, classes are nodes that originate from ontologies. Class data sources are Linked Data sources that are used to create edges in the knowledge graph and thus can connect to other class data sources. Sometimes we want to add data that is not already part of an ontology. In that case, data either be added as an `instance` of an existing ontology class or as its own `owl:class` by being added to the knowledge graph as a `subclass` of an existing `owl:class`.

In [38]:
#edges = LinkedData('resources/edge_source_list.txt', 'resources/resource_info.txt')
#edges.downloads_data_from_url()

In [39]:
edges = LinkedData('resources/edge_source_list.txt', 'resources/resource_info.txt')

In [40]:
edges.parses_resource_file()

In [41]:
edges.source_list

{'chemical-disease': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'chemical-gene': 'resources/edge_data/chemical-gene_CTD_chem_gene_ixns.tsv',
 'chemical-gobp': 'resources/edge_data/chemical-gobp_CTD_chem_go_enriched.tsv',
 'chemical-gocc': 'resources/edge_data/chemical-gocc_CTD_chem_go_enriched.tsv',
 'chemical-gomf': 'resources/edge_data/chemical-gomf_CTD_chem_go_enriched.tsv',
 'chemical-pathway': 'resources/edge_data/chemical-pathway_ChEBI2Reactome_All_Levels.txt',
 'chemical-protein': 'resources/edge_data/chemical-protein_CTD_chem_gene_ixns.tsv',
 'chemical-phenotype': 'resources/edge_data/chemical-disease_CTD_chemicals_diseases.tsv',
 'disease-phenotype': 'resources/edge_data/disease-phenotype_phenotype.hpoa',
 'gene-disease': 'resources/edge_data/gene-disease_curated_gene_disease_associations.tsv',
 'gene-gene': 'resources/edge_data/gene-gene_COMBINED.DEFAULT_NETWORKS.BP_COMBINING.txt',
 'gene-pathway': 'resources/edge_data/gene-pathway_CTD_genes_pathways.

In [42]:
edges.data_files = edges.source_list
edges.generates_source_metadata()


*** Generating Metadata ***



100%|██████████| 39/39 [00:00<00:00, 8941.61it/s]


In [43]:
edges._writes_source_metadata_locally()

100%|██████████| 39/39 [00:00<00:00, 35291.88it/s]


In [44]:
edges.source_list.keys()

dict_keys(['chemical-disease', 'chemical-gene', 'chemical-gobp', 'chemical-gocc', 'chemical-gomf', 'chemical-pathway', 'chemical-protein', 'chemical-phenotype', 'disease-phenotype', 'gene-disease', 'gene-gene', 'gene-pathway', 'gene-phenotype', 'gene-protein', 'gene-rna', 'gobp-pathway', 'pathway-gocc', 'pathway-gomf', 'protein-anatomy', 'protein-catalyst', 'protein-cofactor', 'protein-cell', 'protein-gobp', 'protein-gocc', 'protein-gomf', 'protein-pathway', 'protein-protein', 'rna-anatomy', 'rna-cell', 'rna-protein', 'variant-disease', 'variant-gene', 'variant-phenotype', 'transporter-chemical', 'chemical-molecule', 'chemical-substrate', 'chemical-inhibitor', 'chemical-indication', 'chemical-adr'])

<br>

***

## Create Edge Lists <a class="anchor" id="create-edges"></a>

**Wiki Page:** **[`Data Sources`](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)**

<br>

**Purpose:** The code below will take the dictionaries of processed data described above and use them to create edge lists for each of the edge types specified in the [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt). Each edge list will be appended to a nested dictionary (see details below).

<br>

**Assumptions:**  
1. All `ontology` and `edge` data sources have been downloaded.   

2. All code in the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook has been run. This Notebook contains code needed to generate all mapping, filtering, and label data.

<br>

**Output:** `Master_Edge_List_Dict.json`. Below is an example of what the `Master Edge Dictionary` contains for each processed resource:  
```python
master_edges = {'chemical-disease'  :
                {'source_labels'    : ';MESH_;',
                 'data_type'        : 'class-class',
                 'edge_relation'    : 'RO_0002606',
                 'uri'              : ('http://purl.obolibrary.org/obo/',
                                       'http://purl.obolibrary.org/obo/'),
                 'delimiter'        : '#',
                 'column_idx'       : '1;4',
                 'identifier_maps'  : '0:./MESH_CHEBI_MAP.txt;1:disease-dbxref-map',
                 'evidence_criteria': "5;!=;' ",
                 'filter_criteria'  : 'None',
                 'edge_list'        : ['...']}
```

***

In [45]:
# set-up environment for parallel processing -- even if running program serially these steps are needed
import psutil
cpus = psutil.cpu_count(logical=True)
ray.init()

{'node_ip_address': '10.226.40.229',
 'raylet_ip_address': '10.226.40.229',
 'redis_address': '10.226.40.229:6379',
 'object_store_address': '/tmp/ray/session_2024-06-01_11-59-03_551688_507698/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2024-06-01_11-59-03_551688_507698/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2024-06-01_11-59-03_551688_507698',
 'metrics_export_port': 55378,
 'node_id': '1143f74b55c13b786447f244b523fb0d5c8a20621b5ea714532934fc'}

In [46]:
# combine data sources
combined_edges = dict(edges.data_files, **ont.data_files)
resource_info_loc = './resources/resource_info.txt'

# initialize edge dictionary class
master_edges = CreatesEdgeList(data_files=combined_edges, source_file=resource_info_loc)
master_edges.runs_creates_knowledge_graph_edges(source_file=resource_info_loc, data_files=combined_edges, cpus=cpus)

(pid=748150) Finished Edge: gene-protein (gene = 19324, protein = 19129); 19527 unique edges
(pid=748147) Finished Edge: gobp-pathway (gobp = 479, pathway = 672); 672 unique edges
(pid=748150) Finished Edge: rna-protein (rna = 28220, protein = 18936); 28220 unique edges
(pid=748159) Finished Edge: gene-rna (gene = 25392, rna = 194965); 197370 unique edges
(pid=748146) Finished Edge: gene-disease (gene = 5059, disease = 4484); 13344 unique edges
(pid=748152) Finished Edge: gene-pathway (gene = 10369, pathway = 1860); 107009 unique edges
(pid=748155) Finished Edge: gene-phenotype (gene = 6787, phenotype = 1595); 24695 unique edges
(pid=748163) Finished Edge: chemical-pathway (chemical = 2247, pathway = 2243); 29988 unique edges
(pid=748154) Finished Edge: disease-phenotype (disease = 12100, phenotype = 10129); 444807 unique edges
(pid=748155) Finished Edge: rna-cell (rna = 10052, cell = 4); 33152 unique edges
(pid=748163) Finished Edge: protein-cell (protein = 10044, cell = 128); 75311 u

KeyboardInterrupt: 

(pid=748146) Finished Edge: protein-pathway (protein = 10540, pathway = 2508); 117707 unique edges
(pid=748154) Finished Edge: protein-gomf (protein = 17807, gomf = 4430); 69816 unique edges


**Preview Master Edge Data**  
Generate a table that includes each `edge-type`, its primary `relation`, example identifiers, and count of unique edges.

In [ ]:
# # read in master edge dictionary
master_edges = json.load(open('resources/Master_Edge_List_Dict.json', 'r'))
master_edges.keys()

dict_keys(['chemical-protein', 'gene-disease', 'chemical-disease', 'rna-protein', 'variant-gene', 'chemical-molecule', 'protein-anatomy', 'protein-gocc', 'variant-phenotype', 'disease-phenotype', 'protein-cell', 'gene-rna', 'protein-gobp', 'protein-protein', 'rna-anatomy', 'protein-catalyst', 'chemical-pathway', 'chemical-inhibitor', 'gene-gene', 'transporter-chemical', 'gene-protein', 'chemical-gobp', 'gobp-pathway', 'chemical-gocc', 'protein-pathway', 'pathway-gomf', 'chemical-adr', 'protein-gomf', 'chemical-gene', 'rna-cell', 'chemical-phenotype', 'protein-cofactor', 'gene-pathway', 'chemical-indication', 'pathway-gocc', 'gene-phenotype', 'variant-disease', 'chemical-substrate', 'chemical-gomf'])

In [ ]:
# read in relation data
relation_data = open('./resources/relations_data/RELATIONS_LABELS.txt').readlines()
relation_dict = {x.split('\t')[0]: x.split('\t')[1].strip('\n') for x in relation_data}
len(relation_dict.keys())


685

In [ ]:
relation_dict['DIDEO_00000041'] = 'is substrate of'

In [ ]:
# print basic stats on each resource
edge_data = [[key, master_edges[key]['edge_relation'],
              ', '.join(master_edges[key]['edge_list'][0]),
              len(master_edges[key]['edge_list'])]
             for key in master_edges.keys()]

# convert dict to pandas df for nice printing
df = pandas.DataFrame(edge_data, columns = ['Edge Type', 'Relation', 'Example Edge', 'Unique Edges']) 
df  

,Edge Type,Relation,Example Edge,Unique Edges
0,chemical-protein,RO_0002434,"CHEBI_4592, PR_P07099",72564
1,gene-disease,RO_0003302,"2, MONDO_0010200",13344
2,chemical-disease,RO_0002610,"CHEBI_71223, MONDO_0018229",112863
3,rna-protein,RO_0002513,"ENST00000499023, PR_O43660",28220
4,variant-gene,RO_0002566,"rs367697256, 7476",236121
5,chemical-molecule,RO_0002436,"CHEBI_35553, PR_P10635",393
6,protein-anatomy,RO_0001025,"PR_P04217, UBERON_0001114",26815
7,protein-gocc,RO_0001025,"PR_A0A024RBG1, GO_0005829",82524
8,variant-phenotype,RO_0003302,"rs527236137, HP_0000556",4855
9,disease-phenotype,RO_0002200,"MONDO_0010761, HP_0000510",444807


In [ ]:
df.to_csv('resources/edge_data/edge_counts.tsv', sep='\t', index=False)

<br><br>

***

## Build Knowledge Graph  <a class="anchor" id="build-kg"></a>
**Wiki Pages:**  
- **[`KG-Construction`](https://github.com/callahantiff/PheKnowLator/wiki/KG-Construction)**  
- **[`relations-data`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data)**  
- **[`node-metadata`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata)** 

**Jupyter Notebooks:**  
- [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb)  
[`Ontology_Cleaning.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Ontology_Cleaning.ipynb)  


<br>

**Assumptions:**  
- <u>Construction Approach</u>. If using the `subclass-based` construction approach, please make sure that a `pickled` dictionary mapping each non-ontology data node to an existing ontology class is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/tree/master/resources/knowledge_graphs#construction-method) for additional information).   
- <u>Relations Data</u>. If inverse relation data is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/relations_data` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/relations_data/README.md) for additional information).  
- <u>Node Metadata</u>. If node metadata is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/node_metadata` directory (please see [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/node_data/README.md) for additional information).  
- <u>Decoding OWL Semantics</u>. If decoding OWL-Semantics, please make sure to provide a list of owl:Property types to keep is created and added to the `./resources/knowledge_graph` directory (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information). 

<br>

**Input:** 
- `Master_Edge_List_Dict.json`  
- Directory of relations data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information
- Directory of node data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) for more information

<br>

**Output:** Please see [`Release v2.0.0 Wiki`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0) for access to all generated output files.   
- `Knowledge Graph` (`.owl` and Networkx MultiDiGraph `.pkl`)  
- `Class Instance URI-UUID Map` (if "instance" construction approach)   
- `Triple List - Integer`  
- `Triple List - Identifier`  
- `Node Integer-Identifier Map`  
- `Node Attribute Data`  

<br>

The process to build the knowledge graph is somewhat time consuming and can be broken into the following steps:  

1. Merge Ontologies. See [here](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontologies/README.md) for additional information on how to preprocess the ontologies prior to merging them.    

2. Create Edges. Add edge lists to merged ontologies.  

3. Add Inverse Relations and Node Data. See the [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) Wiki page for details on how to construct these resources.  

4. Filter OWL Semantics. Filter the knowledge graph with the goal of removing all edges that contain entities that are needed to support owl semantics, but are not biologically meaningful (please see [here](https://github.com/callahantiff/PheKnowLator/wiki/OWL-NETS-2.0) for additional information).

5. Save Edge Lists and Node Metadata. Several versions of the knowledge graph are saved, including: the full knowledge graph (`owl` or Networkx MultiDiGraph `pickle`), triple lists (i.e. integer index and identifier labeled edge lists with a dictionary that maps between the integer indices and node identifiers), and a file of metadata (i.e. identifiers, labels, synonyms, and descriptions) for all nodes in the knowledge graph.  

<br>

**‼ IMPORTANT:**  
- The file containing the merged ontologies is quite large and can take up to 30 minutes to read in.  This is not a limitation of the code directly, but rather a function of the [`RDFLib Library`](https://github.com/RDFLib). While there are other ways to read in this data, we maintain reliance on this library as it is the most user-friendly for non-RDF users.   
- If you'd like to include [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) when building the knowledge graph, please hold off on building the knowledge graph until you have generated the node data. For details on how to do this see the [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) section of the `Dependencies` Wiki Page or help for help with generating the data, please see the []() section of the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/notebooks/Data_Preparation.ipynb) Jupyter Notebook.

***


In [ ]:
# specify input arguments
build = 'full'
construction_approach = 'instance'
add_node_data_to_kg = 'yes'
add_inverse_relations_to_kg = 'yes'
decode_owl_semantics = 'yes'
kg_directory_location = './resources/knowledge_graphs'

In [ ]:
%%time
# construct knowledge graphs
##Run sed command here again for PheKnowLator_MergedOntologies.owl
if build == 'partial':
    kg = PartialBuild(construction=construction_approach,
                      node_data=add_node_data_to_kg,
                      inverse_relations=add_inverse_relations_to_kg,
                      decode_owl=decode_owl_semantics,
                      cpus=cpus,
                      write_location=kg_directory_location)
elif build == 'post-closure':
    kg = PostClosureBuild(construction=construction_approach,
                          node_data=add_node_data_to_kg,
                          inverse_relations=add_inverse_relations_to_kg,
                          decode_owl=decode_owl_semantics,
                          cpus=cpus,
                          write_location=kg_directory_location)
else:
    kg = FullBuild(construction=construction_approach,
                   node_data=add_node_data_to_kg,
                   inverse_relations=add_inverse_relations_to_kg,
                   decode_owl=decode_owl_semantics,
                   cpus=cpus,
                   write_location=kg_directory_location)

kg.construct_knowledge_graph()
ray.shutdown()


### Starting Knowledge Graph Build: FULL ###
*** Loading Relations Data ***
Loading and Processing Relation Data
*** Loading Merged Ontologies ***
Merged Ontologies Graph Stats: 19408225 triples, 7111264 nodes, 330 predicates, 834761 classes, 36 individuals, 867 object props, 700 annotation props
*** Loading Node Metadata Data ***
Loading and Processing Node Metadata

Extracting Class and Relation Metadata


100%|██████████| 867/867 [00:00<00:00, 4118.58it/s]


*** Splitting Graph ***
Adding Namespace to BNodes
Creating Logic and Annotation Subsets of Graph


100%|██████████| 2185081/2185081 [26:25<00:00, 1378.60it/s] 


Annotation Assertions (n=13123839 Triples)
Creating Logic Graph (n=6284386 Triples)


...

Depduplicating File: ./resources/knowledge_graphs/PheKnowLator_v3.1.2_full_instance_inverseRelations_OWL_AnnotationsOnly.nt


100%|██████████| 14093577/14093577 [01:02<00:00, 224386.58it/s]


Depduplicating File: ./resources/knowledge_graphs/PheKnowLator_v3.1.2_full_instance_inverseRelations_OWL_LogicOnly.nt


100%|██████████| 26825799/26825799 [02:44<00:00, 163212.37it/s]


Merging Files: ./resources/knowledge_graphs/PheKnowLator_v3.1.2_full_instance_inverseRelations_OWL_AnnotationsOnly.nt and ./resources/knowledge_graphs/PheKnowLator_v3.1.2_full_instance_inverseRelations_OWL_LogicOnly.nt


Loading Full (Logic + Annotation) Graph

Deriving Stats

Processing pkt-namespaced BNodes in Full (Logic + Annotation) graph
Identifying BNodes with Namespace: https://github.com/callahantiff/PheKnowLator/pkt/bnode/
Identifying BNodes
Removing Namespace from BNodes
Finalizing Updated Graph
Identifying BNodes with Namespace: https://github.com/callahantiff/PheKnowLator/pkt/
Identifying BNodes
Removing Namespace from BNodes
Finalizing Updated Graph
Applying OWL API Formatting to Knowledge Graph OWL File
None

Full (Logic + Annotation) Graph Stats: 40919376 triples, 15507789 nodes, 357 predicates, 1269316 classes, 6985184 individuals, 867 object props, 700 annotation props
CPU times: user 17h 20min 50s, sys: 1h 47min 35s, total: 19h 8min 26s
Wall time: 19h 22min 8s


In [ ]:
kg.ontologies

['/home/sanya/PheKnowLator/resources/ontologies/clo_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/chebi_lite_merged_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/chebi_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/ro_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/dideo_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/oae_merged_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/pr_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/po_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/cl_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/so_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/hp_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/mondo_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/uberon_with_imports.owl',
 '/home/sanya/PheKnowLator/resources/ontologies/pw_wi

<br>

***
***

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```